In [69]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio

fig_dir = Path('/bsuhome/zacharykeskinen/uavsar-coherence/figures')

out_dir = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar/')
sites = sorted(list(out_dir.glob('*_v2.nc')))
sites = [s for s in sites if 'full' not in s.stem]

for site in sites:
    full = xr.open_dataset(site)
    
    stem = site.stem.replace('_v2', '')
    uavsars = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar/tiffs').rglob(f'{stem}*.int.grd.tiff')
    concat_dss = []
    for fp in uavsars:
        ds = np.abs(xr.open_dataarray(fp).squeeze('band', drop = True))
        ds = 10 * np.log10(ds)
        ds = ds.rio.reproject_match(full)
        ann = pd.read_csv(list(fp.parent.glob('*.csv'))[0], index_col = 0)
        
        t1, t2 = ann.loc['value', 'start time of acquisition for pass 1'], ann.loc['value', 'start time of acquisition for pass 2']
        # round to day to match heading dates
        t1, t2 = pd.to_datetime(t1).round('d'), pd.to_datetime(t2).round('d')
        # heading = ann.loc['value', 'peg heading']
        heading = int(fp.stem.split('_')[1][:3])
        pol = fp.stem.split('_')[-2][4:]

        ds = ds.expand_dims(heading = [heading])
        ds = ds.expand_dims(time1 = [t1])
        ds = ds.expand_dims(time2 = [t2])
        ds = ds.expand_dims(pol = [pol])

        concat_dss.append(ds) # .chunk()
    ds = xr.combine_by_coords(concat_dss).rename({'band_data': 'signal_amplitude'})
    ds = ds.where((ds > -1e7) & (ds < 1e7))
    ds['time1'] = pd.to_datetime(ds.time1)
    ds['time2'] = pd.to_datetime(ds.time2)
    full['signal_amplitude'] = ds['signal_amplitude']
    
    full.to_netcdf(out_dir.joinpath(stem + '_v3.nc'))